# Basic usage of rocks

How to use <a href="https://rocks.readthedocs.io/en/latest/"><tt>rocks</tt></a> to easily access information on Solar System Objects (SSOs), from the <a href="https://ssp.imcce.fr/webservices/ssodnet/">Web service</a> of <a href="https://ssp.imcce.fr/forms/ssocard">SsODNet</a> (<a href="https://ui.adsabs.harvard.edu/abs/2023A%26A...671A.151B/abstract">Berthier et al., 2023)</a>.

In [1]:
import rocks

Table of Contents
* [Identification of Minor Bodies](#identification)
* [Retrieve best estimate of parameter from ssoCard for one Minor Body](#best-estimate-one)
* [Retrieve best estimate of parameter from ssoCard for many Minor Bodies](#best-estimate-many)
* [Retrieve all estimates of parameter from the datacloud for one Minor Body](#all-estimates)
* [Retrieve best estimates of all parameters for all Minor Body](#bft)

### Identification of Minor Bodies  <a class="anchor" id="identification"></a>

Define targets by their identification, either the IAU number, a designation (including packed designation), or the name. <tt>rocks</tt> deals with lower/upper cases, spaces, etc.

In [2]:
targets = [1, 234, "Pallas", "G!kun||'homdima", "6344 P-L", "J95X00A", "2000 CR105", "1999RQ36"]

In [3]:
# Run the identification
name_num = rocks.id(targets)
print('Names and numbers:')
print(name_num)

# If you only care about the names / numbers
names = [name for name, _ in name_num]
numbers = [number for _, number in name_num]
print("\nNumbers only:")
print(numbers)

Names and numbers:
[('Ceres', 1), ('Barbara', 234), ('Pallas', 2), ("G!kun||'homdima", 229762), ('6344 P-L', nan), ('Biagiomarin', 24850), ('2000 CR105', 148209), ('Bennu', 101955)]

Numbers only:
[1, 234, 2, 229762, nan, 24850, 148209, 101955]


### Retrieve the best estimates for a given SSO from the ssoCard  <a class="anchor" id="best-estimate-one"></a>

See <a href="https://ssp.imcce.fr/webservices/ssodnet/api/ssocard">SsODNet/ssoCard documentation</a> and <a href="https://ui.adsabs.harvard.edu/abs/2023A%26A...671A.151B/abstract">Berthier et al. (2023)</a> for a  complete description of how the best estimates are determined. See also <a href="https://rocks.readthedocs.io/en/latest/cli.html#getting-values"><tt>rocks</tt> documentation</a> for a full description of the python interface.

In [4]:
# Every minor body is a "Rock". Running this command downloads the ssoCard and ingests the data
sso = rocks.Rock(1)

In [5]:
sso.number

1

In [6]:
sso.name  # the asteroid was identified based on the argument passed to rocks.Rock(), '1' in this case

'Ceres'

In [7]:
sso.diameter.value  # accessing parameter values follows a simple scheme

939.4

In [8]:
sso.diameter.error.min, sso.diameter.error.max  # or just sso.diameter.error_ to get the mean of min and max

(-0.4, 0.4)

In [9]:
sso.mass.value  # another parameter, same basic code format

9.384e+20

In [10]:
sso.mass.error_ # to get the average uncertainty instead of the lower and upper 1 $\sigma$.

1e+17

There is also a complete traceability of sources with a bibliography attached to each parameter:

In [11]:
sso.diameter.bibref

[Bibref(path='', doi='10.1126/science.aaf4219', year=2016, title='Dawn arrives at Ceres: Exploration of a small, volatile-rich world', bibcode='2016Sci...353.1008R', shortbib='Russell+2016')]

Like all parameters in rocks, the lower levels can be accessed using the dot-notation.

In [12]:
sso.diameter.bibref.bibcode

['2016Sci...353.1008R']

Putting things together:

In [13]:
print(f'Diameter of ({sso.number}) {sso.name}: {sso.diameter.value} +/-'
      f'({sso.diameter.error.min:.1},{sso.diameter.error.max:.1}) {sso.diameter.unit}'
      f' by {";".join(sso.diameter.bibref.shortbib)}')

Diameter of (1) Ceres: 939.4 +/-(-0.4,0.4) km by Russell+2016


<a href="https://ssp.imcce.fr/webservices/ssodnet/">SsODNet</a> and its <a href="https://rocks.readthedocs.io/en/latest/"><tt>rocks</tt></a> interface propose many parameters. See the <a href="https://ssp.imcce.fr/forms/ssocard/doc"><tt>ssoCard</tt> documentation</a> for a complete list of available parameters, and
<a href="https://rocks.readthedocs.io/en/latest/appendix.html#parameter-names"><tt>rocks</tt> documentation</a> 
to see how to access them.

### Retrieve the best estimates for several SSO from the ssoCard  <a class="anchor" id="best-estimate-many"></a>

See <a href="https://ssp.imcce.fr/webservices/ssodnet/api/ssocard">SsODNet/ssoCard documentation</a> and <a href="https://ui.adsabs.harvard.edu/abs/2023A%26A...671A.151B/abstract">Berthier et al. (2023)</a> for a  complete description of how the best estimates are determined. See also <a href="https://rocks.readthedocs.io/en/latest/cli.html#getting-values"><tt>rocks</tt> documentation</a> for a full description of the python interface.

In [14]:
import time
import numpy as np
import pandas as pd

Let's query multiple ssoCards at once. To speed up the process, they are downloaded asynchronously on the first run and cached on disk. A second run uses the cached data and is much faster.

In [15]:
targets = [1, 234, "Pallas", "G!kun||'homdima", "6344 P-L", "J95X00A", "2000 CR105", "1999RQ36"]

In [16]:
start = time.time()
ssos = rocks.rocks(targets)
print(f"This took {time.time() - start:.3} seconds.")

This took 0.141 seconds.


We will now organize the information into a <a href="https://pandas.pydata.org/">pandas</a> <a href="https://pandas.pydata.org/docs/reference/frame.html">DataFrame</a> to export it. Usually, it is much easier
to use the rocks themselves in your code instead of creating a pandas dataframe.

In [17]:
data = pd.DataFrame(index=range(len(targets)))

for i, sso in enumerate(ssos):

    # Base identification - Always present in ssoCard
    data.loc[i,'num'] = sso.number
    data.loc[i,'name'] = sso.name

    # Osculating elements - Always present in ssoCard
    data.loc[i,'H'] = sso.absolute_magnitude.value
    data.loc[i,'a'] = sso.orbital_elements.semi_major_axis.value
    data.loc[i,'e'] = sso.orbital_elements.eccentricity.value
    data.loc[i,'i'] = sso.orbital_elements.inclination.value

    # Taxonomy - Only add values if present in the ssoCard
    if sso.taxonomy:
        data.loc[i,"taxo_class"] = sso.taxonomy.class_.value
        data.loc[i,"taxo_complex"] = sso.taxonomy.complex.value
        data.loc[i,"taxo_scheme"] = sso.taxonomy.scheme.value
        data.loc[i,"taxo_method"] = sso.taxonomy.technique.value
        data.loc[i,"taxo_range"] = sso.taxonomy.waverange.value
        data.loc[i,"taxo_ref"] = ','.join(sso.taxonomy.bibref.bibcode)

    # Diameter - Only add values if present in the ssoCard
    if sso.diameter:
        data.loc[i,"diameter"] = sso.diameter.value
        data.loc[i,"err_diameter"] = sso.diameter.error_
        data.loc[i,"err_diameter_low"] = sso.diameter.error.min
        data.loc[i,"err_diameter_up"] = sso.diameter.error.max
        data.loc[i,"diameter_method"] = sso.diameter.method[0].name
        data.loc[i,"diameter_ref"] = ','.join(sso.diameter.bibref.bibcode)

In [18]:
data

,num,name,H,a,e,i,taxo_class,taxo_complex,taxo_scheme,taxo_method,taxo_range,taxo_ref,diameter,err_diameter,err_diameter_low,err_diameter_up,diameter_method,diameter_ref
0,1.0,Ceres,3.33,2.767342,0.078884,10.586466,C,C,Mahlke,Spec,VISNIR,2022A&A...665A..26M,939.400,0.400,-0.400,0.400,Rendez-vous with a spacecraft,2016Sci...353.1008R
1,234.0,Barbara,9.10,2.385375,0.245080,15.375318,L,L,Mahlke,Spec,VISNIR,2022A&A...665A..26M,46.300,5.000,-5.000,5.000,"Knitted Occultation, Adaptive-optics, and Ligh...",2015MNRAS.448.3382T
2,2.0,Pallas,4.12,2.769923,0.230163,34.926428,B,B,Mahlke,Spec,VISNIR,2022A&A...665A..26M,512.588,5.336,-5.336,5.336,All-Data Asteroid Model,"2021A&A...654A..56V,2010Icar..205..460C,2020Na..."
3,229762.0,G!kun||'homdima,3.55,73.043150,0.485505,23.389756,NaN,NaN,NaN,NaN,NaN,NaN,609.678,15.912,-15.912,15.912,Stellar Occultation,2019pdss.data....3H
4,NaN,6344 P-L,20.39,2.819759,0.661693,4.677970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,24850.0,Biagiomarin,15.54,2.327661,0.129885,5.726285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,148209.0,2000 CR105,6.30,215.583201,0.795903,22.824297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,101955.0,Bennu,20.69,1.125977,0.203746,6.033844,B,B,Mahlke,Spec,VISNIR,2022A&A...665A..26M,0.488,0.019,-0.019,0.019,Rendez-vous with a spacecraft,2019NatGe..12..247B


### Retrieve all estimates from the literature for a given SSO from the datacloud   <a class="anchor" id="all-estimates"></a>


See <a href="https://ssp.imcce.fr/webservices/ssodnet/api/datacloud">SsODNet/datacloud documentation</a> and <a href="https://ui.adsabs.harvard.edu/abs/2023A%26A...671A.151B/abstract">Berthier et al. (2023)</a> for a  complete description of which parameters are included in the datacloud. See also <a href="https://rocks.readthedocs.io/en/latest/cli.html#getting-values"><tt>rocks</tt> documentation</a> for a full description of the python interface.

Let's retrieve all diameter and albedo measurements of Vesta. The datacloud collections are stored as <a href="https://pandas.pydata.org/">pandas</a> <a href="https://pandas.pydata.org/docs/reference/frame.html">DataFrame</a> in <tt>rocks</tt>:

In [19]:
vesta = rocks.Rock(4, datacloud="diamalbedo")
vesta.diamalbedo

,link,title,shortbib,datasetname,idcollection,resourcename,bibcode,doi,year,id_,number,name,diameter,err_diameter_up,err_diameter_down,albedo,err_albedo_up,err_albedo_down,beaming,err_beaming,emissivity,err_emissivity,selection,method,preferred_albedo,preferred_diameter,preferred
0,None,Diameter and albedo estimates,Tedesco+2002a,None,None,None,2002AJ....123.1056T,None,2002,198107,4,Vesta,468.300,26.700,-26.700,0.423,0.053,-0.053,NaN,NaN,NaN,None,None,STM,False,False,False
1,None,Diameter and albedo estimates,Herald+2019,None,None,None,2019pdss.data....3H,None,2019,198108,4,Vesta,505.368,3.477,-3.477,NaN,NaN,NaN,NaN,NaN,NaN,None,None,OCC,False,False,False
2,None,Diameter and albedo estimates,Drummond+1998,None,None,None,1998Icar..132...80D,None,1998,198109,4,Vesta,507.304,11.865,-11.865,NaN,NaN,NaN,NaN,NaN,NaN,None,None,TE-IM,False,False,False
3,None,Diameter and albedo estimates,Drummond+2008,None,None,None,2008Icar..197..480D,None,2008,198110,4,Vesta,509.980,10.420,-10.420,NaN,NaN,NaN,NaN,NaN,NaN,None,None,TE-IM,False,False,False
4,None,Diameter and albedo estimates,Ryan+2010,None,None,None,2010AJ....140..933R,None,2010,198111,4,Vesta,515.855,19.247,-19.247,0.348,0.026,-0.026,0.842,0.039,NaN,None,None,NEATM,False,False,False
5,None,Diameter and albedo estimates,Alí-Lagoa+2020,None,None,None,2020A&A...638A..84A,None,2020,198112,4,Vesta,520.000,12.000,-6.000,NaN,NaN,NaN,NaN,NaN,0.90,None,None,TPM,False,False,False
6,None,Diameter and albedo estimates,Alí-Lagoa+2020,None,None,None,2020A&A...638A..84A,None,2020,198113,4,Vesta,520.000,21.000,-9.000,NaN,NaN,NaN,NaN,NaN,0.90,None,None,TPM,False,False,False
7,None,Diameter and albedo estimates,Ryan+2010,None,None,None,2010AJ....140..933R,None,2010,198114,4,Vesta,520.367,6.840,-6.840,0.342,0.012,-0.012,NaN,NaN,NaN,None,None,STM,False,False,False
8,None,Diameter and albedo estimates,Usui+2011,None,None,None,2011PASJ...63.1117U,None,2011,198115,4,Vesta,521.740,7.500,-7.500,0.342,0.013,-0.013,NaN,NaN,NaN,None,None,NEATM,False,False,False
9,None,Diameter and albedo estimates,Herald+2019,None,None,None,2019pdss.data....3H,None,2019,198116,4,Vesta,522.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,OCC,False,False,False


The best estimates provided in the ssoCard are based on these collections. The datacloud entries in <tt>rocks</tt> have a <kbd>preferred</kbd> attributes, which are lists containing <kbd>True</kbd> if the corresponding observation is preferred, and <kbd>False</kbd> otherwise. 

We refer to <a href="https://ui.adsabs.harvard.edu/abs/2023A%26A...671A.151B/abstract">Berthier et al. (2023)</a> for an extensive description of the selection of preferred entries. 

In [20]:
ceres = rocks.Rock(1, datacloud='masses')
print("Number of mass estimates for Ceres: ", len(ceres.masses.mass))

Number of mass estimates for Ceres:  38


In [21]:
for i, obs in ceres.masses.iterrows(): 
  mean_error = (obs.err_mass_up + abs(obs.err_mass_down)) / 2
  print(f"[{'X' if obs.preferred else ' '}] {obs.mass:.3e} +- {mean_error:.2e} [{obs.shortbib}, Method: {obs.method}]")

[ ] 8.270e+20 +- 3.78e+19 [Kuzmanoski+1996, Method: DEFLECT]
[ ] 8.730e+20 +- 7.96e+18 [Hilton+1999, Method: DEFLECT]
[ ] 9.040e+20 +- 1.39e+19 [Kova+2012, Method: DEFLECT]
[ ] 9.190e+20 +- 1.41e+19 [Sitarski+1995, Method: DEFLECT]
[ ] 9.290e+20 +- 1.79e+19 [Carpino+1996, Method: DEFLECT]
[ ] 9.290e+20 +- 3.68e+18 [Fienga+2013, Method: EPHEM]
[ ] 9.290e+20 +- 3.84e+18 [Fienga+2014, Method: EPHEM]
[ ] 9.310e+20 +- 6.46e+18 [Konopliv+2011, Method: EPHEM]
[ ] 9.320e+20 +- 9.32e+19 [Folkner+2009, Method: EPHEM]
[ ] 9.348e+20 +- 5.97e+19 [Goffin1991, Method: DEFLECT]
[ ] 9.350e+20 +- 5.57e+18 [Konopliv+2006, Method: DEFLECT]
[ ] 9.350e+20 +- 5.97e+19 [Goffin+2001, Method: DEFLECT]
[ ] 9.350e+20 +- 7.96e+18 [Michalak+2000, Method: DEFLECT]
[ ] 9.383e+20 +- 2.29e+18 [Fienga+2019, Method: EPHEM]
[X] 9.384e+20 +- 1.00e+17 [Russell+2016, Method: SPACE]
[ ] 9.380e+20 +- 2.21e+18 [Viswanathan+2017, Method: EPHEM]
[ ] 9.394e+20 +- 1.31e+18 [Baer+2017, Method: DEFLECT]
[ ] 9.390e+20 +- 1.57e+18 [Pit

### Retrieve all best estimates for all SSOs with the BFT   <a class="anchor" id="bft"></a>

Note: this section requires the `fastparquet` python package to be installed:

`python -m pip install fastparquet`

We store all estimates for all SSOS in a large tabular file, see <a href="https://ssp.imcce.fr/webservices/ssodnet/api/bft">SsODNet/bft documentation</a> and <a href="https://ui.adsabs.harvard.edu/abs/2023A%26A...671A.151B/abstract">Berthier et al. (2023)</a>.

We will download it and open it

In [22]:
import wget
import os

In [23]:
bft_url = 'https://ssp.imcce.fr/data/ssoBFT-latest.parquet'

if not os.path.isfile( 'ssoBFT-latest.parquet' ):
    wget.download(bft_url)

In [ ]:
bft = pd.read_parquet('ssoBFT-latest.parquet' )

In [ ]:
bft

Yes, it is as easy as that :-)
<br><br>

You can make selection in that large corpus of data, e.g., just the inner belt:

In [ ]:
imb = bft['sso_class'] == 'MB>Inner'
print( 'Selected {:,d} inner belt asteroids out of {:,d} SSOs'.format(len(bft[imb]), len(bft)) )

Or more complex selection, based on any combination of columns:

In [ ]:
pristine = (bft['orbital_elements.semi_major_axis.value']>2.82) & \
           (bft['orbital_elements.semi_major_axis.value']<2.96)& \
           (bft['orbital_elements.inclination.value']<5)

Let's make a simple figure with a couple of parameters, using our selections.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

# Plot all objects
ax.scatter( bft['orbital_elements.semi_major_axis.value'],
            bft['orbital_elements.inclination.value'],
            color='gray', s=1, alpha=0.015, rasterized=True, label='All' )

# Only the inner belt
ax.scatter( bft.loc[imb, 'orbital_elements.semi_major_axis.value'],
            bft.loc[imb, 'orbital_elements.inclination.value'],
            s=1, alpha=0.05, rasterized=True, label='IMB' )

# Only the low-inclination of the pristine part of the belt  
ax.scatter( bft.loc[pristine, 'orbital_elements.semi_major_axis.value'],
            bft.loc[pristine, 'orbital_elements.inclination.value'],
            s=1, alpha=0.05, rasterized=True, label='Pristine' )

# Set the axes
ax.legend(loc='upper left')
ax.set_xlim(1.7,3.5)
ax.set_ylim(0,40)
ax.set_xlabel('Semi-major axis / au')
ax.set_ylabel('Inclination / deg.')
